# *fiasco*: Testing the Package--Write Tests
Test the packages ability to build up a single-file HDF5 database from the dataframes that were parsed from the files.

In [1]:
import os
import h5py
import numpy as np
#import pandas
import astropy.units as u
from astropy.utils.console import ProgressBar
import fiasco.io
#import matplotlib.pyplot as plt
import fortranformat

%matplotlib inline

In [2]:
skip_dirs = ['abundance','ip','ancillary_data','continuum','dem','ioneq','masterlist']
all_files = []
for topdir,subdir,files in os.walk(fiasco.defaults['chianti_dbase_root']):
    if np.any([sd in topdir for sd in skip_dirs]):
        continue
    if topdir == fiasco.defaults['chianti_dbase_root']:
        continue
    files = [file for file in files if file[0] != '.']
    all_files += files

Create group from filetype name.

Create dataset for each column in the group.

In [5]:
elvlc_files = [file for file in all_files if os.path.splitext(file)[1] == '.elvlc']

In [ ]:
with ProgressBar(len(elvlc_files),ipython_widget=True,) as progress:
    with h5py.File('/Users/willbarnes/.fiasco/chianti_dbase.h5','a') as hf:
        for file in elvlc_files:
            #print('parsing {}'.format(file))
            parser = fiasco.io.Parser(file)
            try:
                df = parser.parse()
            except NotImplementedError:
                print('Could not parse file {}'.format(file))
                continue
            parser.to_hdf5(hf,df)
            progress.update()

The installed widget Javascript is the wrong version. It must satisfy the semver range ~2.1.4.


In [ ]:
with h5py.File('/Users/willbarnes/.fiasco/chianti_dbase.h5','r') as hf:
    grp = hf['/h/h_1']
    print(grp.name)

In [8]:
df = fiasco.io.Parser('h_1.elvlc').parse()

In [13]:
df

level index,configuration,level label,multiplicity,orbital angular momentum,total angular momentum,observed energy,theoretical energy
,,,,,,1 / cm,1 / cm
int64,str2,str1,int64,str1,float64,float64,float64
1,1s,,2,S,0.5,0.0,0.0
2,2s,,2,S,0.5,82258.956,82303.0
3,2p,,2,P,0.5,82258.921,82303.0
4,2p,,2,P,1.5,82259.287,82303.0
5,3s,,2,S,0.5,97492.224,97544.0
6,3p,,2,P,0.5,97492.213,97544.0
7,3p,,2,P,1.5,97492.321,97544.0
8,3d,,2,D,1.5,97492.321,97544.0


In [14]:
df['configuration'].unit is None

True

In [16]:
print(df.meta['footer'])

%filename: h_1.elvlc
%observed energy levels: Fuhr et al, 1999, NIST Atomic Spectra Database Version 2.0
%produced as part of the Arcetri/Cambridge/NRL 'CHIANTI' atomic data base collaboration
%
%  Ken Dere  May 3 2001

